Concernant le projet 1 je vous demande d'entrainer un GAN qui va apprendre a transformer les images de MNIST en image de SVHN. Une fois que vous avez réussi à faire cela je vous laisse vous débrouiller pour réussir à entrainer un CNN pour qu'il puisse marché sur les images du set de test de SVHN tout en ayant aucune annotations pour entrainer sur SVHN. Vous pouvez considérer que vous avez les images de MNIST avec les annotations.